# Set up

In [1]:
import numpy as np
import pandas as pd

import csv
import pickle

from collections import defaultdict

import requests

from datetime import datetime
import locale # to allow date parsing for dates in Dutch

from collections import Counter

import matplotlib.pyplot as plt

import copy

Endpoint function

In [2]:
# Set base_url of api
base_url = "https://ws.vlpar.be/e/opendata"

def get_endpoint(endpoint: str):
    """
    Return data available at inserted endpoint
    """
    # Make the GET request
    response = requests.get(f"{base_url}{endpoint}")
    
    # Initialize data with a default value
    data = None

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data = response.json()  # Parse JSON response
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

    return data

Interesting fields for questions:
* /schv/lijst: Schriftelijke vragen op basis van id's van de vragen
* /vi/lijst: Lijst van vragen en interpellaties op basis van id's  van de initiatieven

# Obtain written questions

In [3]:
schv_json = get_endpoint("/schv/lijst")

# Inspect data
schv_json

# Parse json into dataframe. Visual inspection showed data is stored in 'items' key
schv_df = pd.DataFrame.from_dict(
    pd.json_normalize(schv_json['items']), 
    orient='columns')

Failed to fetch data. Status code: 400


TypeError: 'NoneType' object is not subscriptable